In [ ]:
#pip install scikit-learn

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

### Análises exploratórias

In [ ]:
tabela = pd.read_csv('../dados_coletados/tabela_modelagem.csv', na_values = '...')

In [ ]:
tabela.info()

In [ ]:
alvo = tabela['Rendimento']
ano = tabela['Ano']
ipcc = tabela.iloc[:,11:]

#### Distribuição dos dados 'alvo'

In [ ]:
sns.histplot(alvo)

In [ ]:
alvo.mean()

In [ ]:
alvo.quantile(np.arange(0, 1.1, 0.1))

In [ ]:
sns.histplot(alvo ** .5)

In [ ]:
alvo = alvo**.5

#### Rendimento vs Ano

In [ ]:
sns.scatterplot(x = ano, y = alvo)

In [ ]:
maiores_areas = alvo.loc[alvo > 62]
ma_index = maiores_areas.index

sns.scatterplot(x = ano.loc[ma_index], y = alvo.loc[ma_index])

In [ ]:
tabela.loc[maiores_areas.index, 
           ['Municipio', 'Ano', 'Estado', 'Area_total', 'Toneladas', 'Rendimento']]

##### Definir número aleatório para ser semente nos procedimentos de aleatorização

In [ ]:
#Apenas uma vez para obter um número aleatório para a semente a ser utilizada nos modelos,
#permitindo assim a replicação dos resultados obtidos.
#from random import randint
#randint(1,10000)

In [ ]:
#resultado dos procedimentos acima foi 5744, e esse será utilizado como seed em todo o trabalho
seed = 5744

In [ ]:
amostra_municipios = tabela.sample(25, random_state = seed)['Municipio'].unique()
amostra_municipios

In [ ]:
amostra_teste = tabela[tabela['Municipio'].isin(amostra_municipios)]
for i in amostra_municipios:
    cidade = amostra_teste[amostra_teste['Municipio'] == i]
    cidade_index = cidade.index
    sns.scatterplot(x = ano.loc[cidade_index], y = alvo.loc[cidade_index])

#### Atributos (variáveis climáticas)

In [ ]:
sc = StandardScaler()
ipcc_sc = sc.fit_transform(ipcc)
ipcc_sc

In [ ]:
ipcc_sc = pd.DataFrame(ipcc_sc)
ipcc_sc.columns = ipcc.columns
ipcc_sc

##### Correlação entre atributos climáticos

In [ ]:
ipcc_cm = ipcc_sc.corr().abs()
display (ipcc_cm)

In [ ]:
ipcc_cm = ipcc_sc.corr().abs()
cor_list = ipcc_cm.unstack().sort_values(kind = 'quick', ascending = False)
cor_list[(cor_list >= 0.85) & (cor_list < 1)]

##### PCA

In [ ]:
pca = PCA(n_components = 5)
ipcc_pca = pca.fit_transform(ipcc_sc)
print('Variancia explicada:  ', pca.explained_variance_)
print('Porcentagem do total: ', pca.explained_variance_ratio_)
print('Porcentagem acumulada:', pca.explained_variance_ratio_.cumsum())

In [ ]:
ipcc_pca_eixos = pd.DataFrame(data = ipcc_pca, columns = ['PCA_1', 'PCA_2', 'PCA_3', 'PCA_4', 'PCA_5'])
savedir = '../resultados/exploratorias'
if not os.path.exists(savedir):
    os.makedirs(savedir)
ipcc_pca_eixos.to_csv(savedir + '/ipcc_pca_eixos.csv', index = False)

#### PCA Loadings

In [ ]:
ipcc_pca_loadings = pd.DataFrame(data = pca.components_.T,
                                 columns=['PCA_1', 'PCA_2', 'PCA_3', 'PCA_4', 'PCA_5'],
                                 index = ipcc.columns)
#Destacar os atributos mais importantes para cada componente 
ipcc_pca_loadings.style.apply(lambda x:[
    "background: blue" if abs(v) > abs(x).quantile(0.85) else "" for v in x],axis = 1)

#### Salvar loadings em aquivo

In [ ]:
if not os.path.exists(savedir):
    os.makedirs(savedir)
ipcc_pca_loadings.to_csv(savedir + '/ipcc_pca_loadings.csv', index = True)

#### Biplot PCA1 vs PCA2

In [ ]:
xmax = max(ipcc_pca_eixos['PCA_1'].to_list())
ymax = max(ipcc_pca_eixos['PCA_2'].to_list())
eixo_x =  [i/xmax for i in ipcc_pca_eixos['PCA_1'].to_list()]
eixo_y = [i/xmax for i in ipcc_pca_eixos['PCA_2'].to_list()]
sns.scatterplot(x = eixo_x,
                y = eixo_y,
                legend = True,
                color = 'lightblue')
for i in range(pca.components_[0:2, :].T.shape[0]):
    plt.arrow(0,
              0,
              pca.components_[0:2, :].T[i,0],
              pca.components_[0:2, :].T[i,1],
              color = 'darkblue',
              alpha = 0.5)
    plt.text(pca.components_[0:2, :].T[i,0],
             pca.components_[0:2, :].T[i,1],
             ipcc.columns[i],
             color = 'brown',
             ha = 'center',
             va = 'center')

#### Atributos vs Alvo

In [ ]:
# PCA 1
sns.scatterplot(x = ipcc_pca_eixos['PCA_1'], y = alvo)

In [ ]:
# PCA 2
sns.scatterplot(x = ipcc_pca_eixos['PCA_2'], y = alvo)

In [ ]:
# PCA 3
sns.scatterplot(x = ipcc_pca_eixos['PCA_3'], y = alvo)

In [ ]:
# PCA 4
sns.scatterplot(x = ipcc_pca_eixos['PCA_4'], y = alvo)

In [ ]:
# PCA 5
sns.scatterplot(x = ipcc_pca_eixos['PCA_5'], y = alvo)